In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class DCGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/mnist_%d.png" % epoch)
        plt.close()



Using TensorFlow backend.


In [2]:
dcgan = DCGAN()
dcgan.train(epochs=4000, batch_size=32, save_interval=50)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
__________

/home/ankish1/anaconda3/envs/tensor/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 1.137276, acc.: 40.62%] [G loss: 0.656750]
1 [D loss: 0.556322, acc.: 71.88%] [G loss: 1.117728]
2 [D loss: 0.387165, acc.: 87.50%] [G loss: 1.402214]
3 [D loss: 0.307202, acc.: 87.50%] [G loss: 1.199606]
4 [D loss: 0.210917, acc.: 95.31%] [G loss: 0.616006]
5 [D loss: 0.182177, acc.: 96.88%] [G loss: 0.624122]
6 [D loss: 0.145014, acc.: 100.00%] [G loss: 0.582059]
7 [D loss: 0.221942, acc.: 90.62%] [G loss: 0.630450]
8 [D loss: 0.335021, acc.: 81.25%] [G loss: 0.749261]
9 [D loss: 0.850522, acc.: 54.69%] [G loss: 1.612944]
10 [D loss: 1.365664, acc.: 20.31%] [G loss: 2.155058]
11 [D loss: 0.794306, acc.: 50.00%] [G loss: 2.176868]
12 [D loss: 0.617244, acc.: 64.06%] [G loss: 1.859275]
13 [D loss: 0.421939, acc.: 81.25%] [G loss: 2.127039]
14 [D loss: 0.703928, acc.: 67.19%] [G loss: 1.786264]
15 [D loss: 0.571504, acc.: 70.31%] [G loss: 2.188153]
16 [D loss: 0.564196, acc.: 70.31%] [G loss: 1.757660]
17 [D loss: 0.443160, acc.: 79.69%] [G loss: 1.068786]
18 [D loss: 0.30785

151 [D loss: 1.017209, acc.: 39.06%] [G loss: 1.045700]
152 [D loss: 0.935842, acc.: 40.62%] [G loss: 1.224373]
153 [D loss: 0.911621, acc.: 37.50%] [G loss: 1.112087]
154 [D loss: 0.768496, acc.: 50.00%] [G loss: 0.971529]
155 [D loss: 0.865795, acc.: 42.19%] [G loss: 1.103540]
156 [D loss: 0.830509, acc.: 46.88%] [G loss: 1.005004]
157 [D loss: 0.968703, acc.: 32.81%] [G loss: 0.967768]
158 [D loss: 0.836204, acc.: 51.56%] [G loss: 1.166581]
159 [D loss: 0.818126, acc.: 45.31%] [G loss: 1.230663]
160 [D loss: 0.821949, acc.: 46.88%] [G loss: 1.093097]
161 [D loss: 0.831430, acc.: 53.12%] [G loss: 1.108867]
162 [D loss: 0.856753, acc.: 39.06%] [G loss: 1.053267]
163 [D loss: 0.829612, acc.: 46.88%] [G loss: 1.228042]
164 [D loss: 0.880464, acc.: 45.31%] [G loss: 1.252505]
165 [D loss: 0.828991, acc.: 48.44%] [G loss: 1.189121]
166 [D loss: 0.813288, acc.: 50.00%] [G loss: 1.111261]
167 [D loss: 0.905372, acc.: 48.44%] [G loss: 1.093288]
168 [D loss: 0.759362, acc.: 51.56%] [G loss: 1.

298 [D loss: 0.819165, acc.: 45.31%] [G loss: 1.045954]
299 [D loss: 0.929616, acc.: 37.50%] [G loss: 1.052830]
300 [D loss: 0.729958, acc.: 45.31%] [G loss: 0.688777]
301 [D loss: 0.808094, acc.: 43.75%] [G loss: 0.954473]
302 [D loss: 0.708508, acc.: 56.25%] [G loss: 0.940143]
303 [D loss: 0.905687, acc.: 43.75%] [G loss: 0.912776]
304 [D loss: 0.803763, acc.: 48.44%] [G loss: 1.009754]
305 [D loss: 0.687006, acc.: 56.25%] [G loss: 0.887255]
306 [D loss: 0.783880, acc.: 46.88%] [G loss: 0.875073]
307 [D loss: 0.684416, acc.: 54.69%] [G loss: 1.102211]
308 [D loss: 0.846807, acc.: 46.88%] [G loss: 0.878108]
309 [D loss: 0.808546, acc.: 45.31%] [G loss: 1.137127]
310 [D loss: 0.724991, acc.: 53.12%] [G loss: 1.111190]
311 [D loss: 0.757755, acc.: 48.44%] [G loss: 1.176083]
312 [D loss: 0.722776, acc.: 62.50%] [G loss: 0.937788]
313 [D loss: 0.738115, acc.: 59.38%] [G loss: 1.208601]
314 [D loss: 0.780646, acc.: 53.12%] [G loss: 0.993832]
315 [D loss: 0.726791, acc.: 57.81%] [G loss: 1.

...

3407 [D loss: 0.778256, acc.: 51.56%] [G loss: 0.937460]
3408 [D loss: 0.606478, acc.: 71.88%] [G loss: 1.273277]
3409 [D loss: 0.722180, acc.: 48.44%] [G loss: 0.871481]
3410 [D loss: 0.586035, acc.: 70.31%] [G loss: 0.964015]
3411 [D loss: 0.657576, acc.: 64.06%] [G loss: 0.941264]
3412 [D loss: 0.762611, acc.: 54.69%] [G loss: 0.974820]
3413 [D loss: 0.628519, acc.: 57.81%] [G loss: 1.023474]
3414 [D loss: 0.592050, acc.: 70.31%] [G loss: 1.078353]
3415 [D loss: 0.692675, acc.: 53.12%] [G loss: 1.096292]
3416 [D loss: 0.652198, acc.: 59.38%] [G loss: 1.034567]
3417 [D loss: 0.596736, acc.: 62.50%] [G loss: 1.121480]
3418 [D loss: 0.684854, acc.: 62.50%] [G loss: 1.111244]
3419 [D loss: 0.602100, acc.: 70.31%] [G loss: 1.158760]
3420 [D loss: 0.552459, acc.: 73.44%] [G loss: 1.089081]
3421 [D loss: 0.661043, acc.: 62.50%] [G loss: 1.213470]
3422 [D loss: 0.538061, acc.: 81.25%] [G loss: 0.992129]
3423 [D loss: 0.742735, acc.: 53.12%] [G loss: 0.974293]
3424 [D loss: 0.676300, acc.: 5

3551 [D loss: 0.655364, acc.: 59.38%] [G loss: 1.032794]
3552 [D loss: 0.713863, acc.: 56.25%] [G loss: 0.896533]
3553 [D loss: 0.631661, acc.: 64.06%] [G loss: 1.199343]
3554 [D loss: 0.589675, acc.: 70.31%] [G loss: 1.008291]
3555 [D loss: 0.769396, acc.: 42.19%] [G loss: 0.987561]
3556 [D loss: 0.639231, acc.: 59.38%] [G loss: 1.089777]
3557 [D loss: 0.607930, acc.: 67.19%] [G loss: 1.079244]
3558 [D loss: 0.725245, acc.: 51.56%] [G loss: 0.960419]
3559 [D loss: 0.584440, acc.: 67.19%] [G loss: 1.070485]
3560 [D loss: 0.628363, acc.: 70.31%] [G loss: 1.162912]
3561 [D loss: 0.641200, acc.: 64.06%] [G loss: 0.969180]
3562 [D loss: 0.715025, acc.: 48.44%] [G loss: 1.010337]
3563 [D loss: 0.545790, acc.: 76.56%] [G loss: 1.209167]
3564 [D loss: 0.672516, acc.: 51.56%] [G loss: 1.088245]
3565 [D loss: 0.594536, acc.: 62.50%] [G loss: 1.117132]
3566 [D loss: 0.667379, acc.: 54.69%] [G loss: 0.927597]
3567 [D loss: 0.617587, acc.: 62.50%] [G loss: 1.065910]
3568 [D loss: 0.651270, acc.: 6

3698 [D loss: 0.611663, acc.: 68.75%] [G loss: 0.893957]
3699 [D loss: 0.581496, acc.: 71.88%] [G loss: 1.189189]
3700 [D loss: 0.746693, acc.: 54.69%] [G loss: 0.950536]
3701 [D loss: 0.704162, acc.: 54.69%] [G loss: 1.064012]
3702 [D loss: 0.583557, acc.: 67.19%] [G loss: 1.252403]
3703 [D loss: 0.661173, acc.: 57.81%] [G loss: 1.098256]
3704 [D loss: 0.671777, acc.: 60.94%] [G loss: 1.269342]
3705 [D loss: 0.614229, acc.: 68.75%] [G loss: 1.081005]
3706 [D loss: 0.576821, acc.: 73.44%] [G loss: 1.036194]
3707 [D loss: 0.618001, acc.: 65.62%] [G loss: 1.082564]
3708 [D loss: 0.590677, acc.: 70.31%] [G loss: 1.193377]
3709 [D loss: 0.635146, acc.: 67.19%] [G loss: 1.102583]
3710 [D loss: 0.637077, acc.: 65.62%] [G loss: 1.070168]
3711 [D loss: 0.768193, acc.: 57.81%] [G loss: 1.022516]
3712 [D loss: 0.543543, acc.: 73.44%] [G loss: 1.004354]
3713 [D loss: 0.603276, acc.: 75.00%] [G loss: 1.184624]
3714 [D loss: 0.627112, acc.: 67.19%] [G loss: 1.049429]
3715 [D loss: 0.620091, acc.: 7

3846 [D loss: 0.593197, acc.: 65.62%] [G loss: 0.963581]
3847 [D loss: 0.686994, acc.: 57.81%] [G loss: 0.876382]
3848 [D loss: 0.598053, acc.: 67.19%] [G loss: 0.799501]
3849 [D loss: 0.560580, acc.: 73.44%] [G loss: 1.092623]
3850 [D loss: 0.611995, acc.: 62.50%] [G loss: 1.203829]
3851 [D loss: 0.712003, acc.: 53.12%] [G loss: 0.925526]
3852 [D loss: 0.554398, acc.: 71.88%] [G loss: 0.988476]
3853 [D loss: 0.702618, acc.: 50.00%] [G loss: 1.208844]
3854 [D loss: 0.620489, acc.: 67.19%] [G loss: 0.928069]
3855 [D loss: 0.643082, acc.: 64.06%] [G loss: 1.010096]
3856 [D loss: 0.706437, acc.: 60.94%] [G loss: 0.939954]
3857 [D loss: 0.556659, acc.: 71.88%] [G loss: 1.115760]
3858 [D loss: 0.622255, acc.: 59.38%] [G loss: 1.159411]
3859 [D loss: 0.656689, acc.: 57.81%] [G loss: 1.024287]
3860 [D loss: 0.766126, acc.: 51.56%] [G loss: 1.062427]
3861 [D loss: 0.605561, acc.: 65.62%] [G loss: 1.090569]
3862 [D loss: 0.631300, acc.: 68.75%] [G loss: 1.120954]
3863 [D loss: 0.585808, acc.: 7

3990 [D loss: 0.654615, acc.: 56.25%] [G loss: 1.076865]
3991 [D loss: 0.621876, acc.: 68.75%] [G loss: 1.005944]
3992 [D loss: 0.668703, acc.: 64.06%] [G loss: 1.013757]
3993 [D loss: 0.651044, acc.: 53.12%] [G loss: 1.099780]
3994 [D loss: 0.568256, acc.: 67.19%] [G loss: 1.131080]
3995 [D loss: 0.570890, acc.: 68.75%] [G loss: 1.069782]
3996 [D loss: 0.607511, acc.: 67.19%] [G loss: 1.061691]
3997 [D loss: 0.651387, acc.: 60.94%] [G loss: 0.905136]
3998 [D loss: 0.669859, acc.: 57.81%] [G loss: 1.030931]
3999 [D loss: 0.711202, acc.: 54.69%] [G loss: 0.965884]
